# 基础内容
## 了解网页结构
### 什么是HTML
HTML 是一种浏览器(Chrome, Safari, IE, Firefox等)看得懂的语言, 浏览器能将这种语言转换成我们用肉眼看到的网页。我们要做的就是从其中爬取出我们需要的信息。
### 还有CSS和JavaScript
构成多彩、多功能网页组件。
### 网页基本组成
在HTML中，有许多tag，其中包裹着基本上所有的实体内容。tag主要分成两个部分，`header`和`body`。在`header`中，存放一些网页的元信息，如`title`，这些信息不会被浏览器显示出来。而在`body`中存放的才是我们能够在浏览器上看到的内容，比如：`<h1></h1>`就是主标题，`<h></h>`就是一个段落，`<a></a>`就是链接。

**爬虫想要做的就是根据这些 tag 来找到合适的信息。**

## 爬网页
利用Python爬取网页的基本信息，并打印出这个网页HTML的source code：

In [2]:
from urllib.request import urlopen

# url = 'https://www.baidu.com/'
url = "https://morvanzhou.github.io/static/scraping/basic-structure.html"
# 网页中存在中文，需要在read()完之后使用decode()来进行转码
html = urlopen(url).read().decode('utf-8')
print(html)

<!DOCTYPE html>
<html lang="cn">
<head>
	<meta charset="UTF-8">
	<title>Scraping tutorial 1 | 莫烦Python</title>
	<link rel="icon" href="https://morvanzhou.github.io/static/img/description/tab_icon.png">
</head>
<body>
	<h1>爬虫测试1</h1>
	<p>
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
		<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	</p>

</body>
</html>


通过分析网页HTML可以知道，合理地利用tag的name属性可以快速提取到我们需要的信息。
## 匹配网页内容
### 利用正则表达式
简单的网页匹配通过正则表达式就可以搞定。

In [3]:
import re
# 获取标题的内容
res = re.findall(r"<title>(.+?)</title>", html)
print("\nPage title is: ", res[0])


Page title is:  Scraping tutorial 1 | 莫烦Python


In [4]:
# 获取章节的内容
res = re.findall(r"<p>(.*?)</p>", html, flags=re.DOTALL)
print("\nPage paragraph is: ", res[0])


Page paragraph is:  
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
		<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	


实际应用中比较常用的时：找出一个网页上的所有链接：

In [5]:
res = re.findall(r'href="(.*?)"', html)
print("\nAll links: ", res)


All links:  ['https://morvanzhou.github.io/static/img/description/tab_icon.png', 'https://morvanzhou.github.io/', 'https://morvanzhou.github.io/tutorials/data-manipulation/scraping/']


### BeautifulSoup
复杂一点的网页就需要其他工具的帮助。首先，我们先梳理一下爬网页的流程：
+ 找到要爬网页的网址
+ 使用Python登录上这个网址（`urlopen()`等）
+ 读取网页信息（`read()等`）
+ 将读取的信息放入BeautifulSoup
+ 使用BeautifulSoup选取tag信息等（代替正则表达式）

In [6]:
from bs4 import BeautifulSoup
# 推荐使用lxml解析器
soup = BeautifulSoup(html, features='lxml')
print(soup.h1)
print('\n',soup.p)

<h1>爬虫测试1</h1>

 <p>
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	</p>


当网页中有多个相同的tag时，可以通过`find_all()`找到所有的选项。又注意到，链接其实是存在于`<a href="link">`里面，因此可以看作是tag`<a>`的一个属性，可以像字典一样进行读取。

In [7]:
all_href = soup.find_all("a")
all_href = [l['href'] for l in all_href]
print('\n', all_href)


 ['https://morvanzhou.github.io/', 'https://morvanzhou.github.io/tutorials/data-manipulation/scraping/']


#### bs4加上CSS
CSS的主要作用就是装饰HTML，使页面内容更加丰富。
##### CSS的Class
在爬虫中，我们主要了解CSS的Class，CSS在装饰每一个网页部件的时候，都会给它一个名字，而且一个类型的部件，名字都可以一样。利用这一特性，我们可以快速找到我们想要的内容。

In [8]:
url = "https://morvanzhou.github.io/static/scraping/list.html"
html = urlopen(url).read().decode("utf-8")
print(html)

<!DOCTYPE html>
<html lang="cn">
<head>
	<meta charset="UTF-8">
	<title>爬虫练习 列表 class | 莫烦 Python</title>
	<style>
	.jan {
		background-color: yellow;
	}
	.feb {
		font-size: 25px;
	}
	.month {
		color: red;
	}
	</style>
</head>

<body>

<h1>列表 爬虫练习</h1>

<p>这是一个在 <a href="https://morvanzhou.github.io/" >莫烦 Python</a> 的 <a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/" >爬虫教程</a>
	里无敌简单的网页, 所有的 code 让你一目了然, 清晰无比.</p>

<ul>
	<li class="month">一月</li>
	<ul class="jan">
		<li>一月一号</li>
		<li>一月二号</li>
		<li>一月三号</li>
	</ul>
	<li class="feb month">二月</li>
	<li class="month">三月</li>
	<li class="month">四月</li>
	<li class="month">五月</li>
</ul>

</body>
</html>


##### 按Class匹配

In [9]:
soup = BeautifulSoup(html, features='lxml')

month = soup.find_all('li', {"class": "month"})
for m in month:
    print(m.get_text())

一月
二月
三月
四月
五月


#### bs4加上re
例如：找出一个网页中的全部jpg格式图片的链接。通过观察可以知道，图片链接都藏在`<img src="link">`中，那么利用BeautifulSoup可以很快速找到，再通过正则表达式可以快速过滤掉其他格式的图片。

In [12]:
import re
url = "https://morvanzhou.github.io/static/scraping/table.html"
html = urlopen(url).read().decode('utf-8')

soup = BeautifulSoup(html, features='lxml')

img_links = soup.find_all("img", {"src": re.compile('.*?\.jpg')})
for link in img_links:
    print(link['src'])

https://morvanzhou.github.io/static/img/course_cover/tf.jpg
https://morvanzhou.github.io/static/img/course_cover/rl.jpg
https://morvanzhou.github.io/static/img/course_cover/scraping.jpg


# 如何添加代理
## 使用proxy的步骤
1. 设置代理地址：`proxy = {'http': ''}`
2. 创建ProxyHeader：`proxyHeader = request.ProxyHandler(proxy)`
3. 创建Opener：`opener = request.build_opener(proxyHeader)`
4. 安装Opener：`request.install_opener(opener)`

In [1]:
from urllib import request
# 设置代理地址
proxy = {'http':'52.187.162.198:3128'}
# 创建ProxyHeader
proxyHeader = request.ProxyHandler(proxy)
# 创建Opener
opener = request.build_opener(proxyHeader)
# 安装Opener
request.install_opener(opener)
# 然后剩下的就跟正常使用差不多，只不过此时的request已经是绑定了代理之后的request
url = 'https://www.ncbi.nlm.nih.gov/pmc'
req = request.Request(url)
response = request.urlopen(req)
# print(response.read().decode())

# 小例子: 爬网页
我们去爬取PMC数据库中的内容。观察网页源代码可以看到，每页20条检索结果，均藏在`<div class="rprt">`下的`<div class="rslt">`中，其中，`<div class="title">`中的tag`<a>`存放着搜索结果，即文献名，其属性`ref="index"`存放对应的链接。

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import random

base_url = "https://www.ncbi.nlm.nih.gov/pmc"
his = ["/?term=diabetes"]

url = base_url + his[-1]

html = urlopen(url).read()
soup = BeautifulSoup(html, 'lxml')
result = soup.find_all('div', {"class": "rslt"})

for r in result:
    print(r)

# 使用requests
注意，获取网页的方式是不一样的，常用的方式有GET和POST，一般情况下，前者注意用来打开网页，不往服务器传数据，而后者常用于登录账号，搜索内容，上传文件/图片，往服务器传数据等。
## requests get请求
通过观察地址栏的规律，可以很快利用requests可以模拟网页上的搜索。

In [13]:
import requests
import webbrowser
# 以字典存储搜索关键字
param = {"wd": "Python文档"}
r = requests.get("http://www.baidu.com/s", params=param)
print(r.url)
# 利用webbrowser模块打开浏览器观察搜索结果
webbrowser.open(r.url)

http://www.baidu.com/s?wd=Python%E6%96%87%E6%A1%A3


True

## requests post请求
注意POST与GET的区别，POST以表单的形式将数据加密提交给网页，而GET提交的信息通常能反应在URL上。通常我们需要注意三个内容：
+ Request URL: POST要使用的URL, 注意不是我们填表时的URL, 
+ Request Method: POST,
+ Form Data: 即我们提交的内容.

In [14]:
data = {'firstname': 'Focusxy', 'lastname': 'Hu'}
r = requests.post('http://pythonscraping.com/files/processing.php', data=data)
print(r.text)

Hello there,  !


## 上传照片
需要注意: 
+ 观察URL的变化, 传照片前后一般是有变化的,
+ 观察提交按钮的name, 加入Python字典的key,
+ 确定python字典的值.

In [15]:
file = {'uploadFile': open('./steel.jpg', 'rb')}
r = requests.post('http://pythonscraping.com/files/processing2.php', files=file)
print(r.text)

uploads/
Sorry, there was an error uploading your file.


## 模拟登录
重要的三点:
+ Request URL,
+ Form Data中的用户名和密码,
+ Cookies. Cookies中存储的是用户的登录信息.

通过`requests.post + payload`将用户信息发给服务器, 返回的对象中会有生成的cookies信息, 我们可以将cookies传入到get请求, 这样就可以以登录的名义来访问get页面了.

In [2]:
payload = {'username': 'Morvan', 'password': 'password'}
r = requests.post('http://pythonscraping.com/pages/cookies/welcome.php', data=payload)
print(r.cookies.get_dict())

r = requests.get('http://pythonscraping.com/pages/cookies/profile.php', cookies=r.cookies)
# print(r.text)

NameError: name 'requests' is not defined

## 使用Session登录
使用session可以不用反复向get中传入cookies, 因为session内部已经有了之前的cookies了.

In [3]:
session = requests.Session()
payload = {'username': 'Morvan', 'password': 'password'}
r = session.post('http://pythonscraping.com/pages/cookies/welcome.php', data=payload)
print(r.cookies.get_dict())

r = session.get("http://pythonscraping.com/pages/cookies/profile.php")
# print(r.text)

NameError: name 'requests' is not defined

# 下载文件
需要观察网站源码找出文件的下载地址, 注意是完整的地址. 

In [19]:
import os
# 建立一个文件夹专门用来存储图片
os.makedirs('./img/', exist_ok=True)

IMAGE_URL = "https://morvanzhou.github.io/static/img/description/learning_step_flowchart.png"

## 使用urlretrieve
urllib模块中提供了一个使用非常简单的下载功能urlretrieve, 传入下载地址`IMAGE_URL`和要存放的文件路径就可以了.

In [20]:
from urllib.request import urlretrieve
urlretrieve(IMAGE_URL, './img/image1.png')

('./img/image1.png', <http.client.HTTPMessage at 0x112972f60>)

## 使用request
使用requests模块下载文件, 代码稍长一些, 但是可以更有效率地下载大文件.

In [21]:
import requests
r = requests.get(IMAGE_URL)
with open('./img/image2.png', 'wb') as f:
    f.write(r.content)

通过`r.iter_content(chunk_size)`可以控制chunk的大小, 使文件一个chunk一个chunk的下载, 不必全部下载完全才能保存.

In [22]:
# stream loading
r = requests.get(IMAGE_URL, stream=True)
with open('./img/image3.png', 'wb') as f:
    for chunk in r.iter_content(chunk_size=32):
        f.write(chunk)

# 小例子: 下载
待完善。

# 效率问题
待完善。
## 多线程
采用分布式爬虫可以大大节省程序的运行时间. 根据爬虫的流程, 可以定义两个功能, 一个是用来爬取网页的crawl, 一个是解析网页的parse. 通过这两部分的合作, 加上剔除重复的网址, 基本上可以快速爬取完我们需要的内容.

In [1]:
import multiprocessing as mp
import time
from urllib.request import urlopen, urljoin
from bs4 import BeautifulSoup
import re

base_url = 'https://morvanzhou.github.io/'

def crawl(url):
    response = urlopen(url)
    return response.read().decode() # 网页中有中文内容要加decode()

def parse(html):
    soup = BeautifulSoup(html, 'lxml')
    urls = soup.find_all('a', {"href": re.compile('')})
    title = soup.find()
    
    # 利用Python自带的set去重
    page_urls = set(urljoin(base_url, url['href']) for url in urls)
    url = soup.find('meta', {'property': ''})['content']
    
    return title, page_urls, url

# 定义两个set, 用来搜集爬过的和还没爬过的
unseen = set([base_url,])
seen = set()

SyntaxError: invalid character in identifier (<ipython-input-1-c557e01133d9>, line 15)

# Ajax动态网页
Ajax可以在不重新加载整个网页的基础上，对网页的部分进行更新。这样看上去网页的URL没有变化，那么如何爬取更新的内容呢？

（**注意**：网页URL没有变化不一定就是采用了ajax技术。采用ajax技术的网页，在跳转下一页或者加载更多内容时，地址栏没有刷新。）

Ajax技术的核心是`XMLHttpRequest`对象（简称XHR），可以通过使用XHR对象获取到服务器的数据，然后再通过DOM将数据插入到页面中呈现。虽然名字中包含XML，但Ajax通讯与数据格式无关，所以我们的数据格式可以是XML或JSON等格式。
## 如何爬取ajax动态加载的网页
